# Anomaly (supervised) Detection

In [1]:
%pip install pandas numpy matplotlib scikit-learn catboost catboost-widget

ERROR: Could not find a version that satisfies the requirement catboost-widget (from versions: none)
ERROR: No matching distribution found for catboost-widget
Note: you may need to restart the kernel to use updated packages.


# Load Data

In [2]:
import pandas as pd
df = pd.read_csv('ai4i2020.csv')

In [3]:
df.columns

Index(['UDI', 'Product ID', 'Type', 'Air temperature [K]',
       'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]',
       'Tool wear [min]', 'Machine failure', 'TWF', 'HDF', 'PWF', 'OSF',
       'RNF'],
      dtype='object')

In [4]:
cat_features=['Product ID', 'Type']
X_features= cat_features +  ['Air temperature [K]',
       'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]',
       'Tool wear [min]']
y_features=[ 'TWF', 'HDF', 'PWF', 'OSF', 'RNF']

In [5]:
X=df[X_features]
y=df[y_features]

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# EDA

In [7]:
df['Machine failure'].value_counts()

Machine failure
0    9661
1     339
Name: count, dtype: int64

In [8]:
df[y_features].value_counts()

TWF  HDF  PWF  OSF  RNF
0    0    0    0    0      9652
     1    0    0    0       106
     0    1    0    0        80
          0    1    0        78
1    0    0    0    0        42
0    0    0    0    1        18
          1    1    0        11
     1    0    1    0         6
          1    0    0         3
1    0    0    1    0         2
               0    1         1
          1    1    0         1
Name: count, dtype: int64

# Model

In [9]:
from catboost import CatBoostClassifier
from sklearn import metrics
def train_and_test(on, X_train, y_train, X_val, y_val, cat_features,iterations=300):

    _y_val = y_val[on].values.flatten()
    _y_train = y_train[on].values.flatten()
    
    clf = CatBoostClassifier(
        iterations=iterations,
        random_seed=42,
        loss_function='Logloss',
        cat_features=cat_features,
        auto_class_weights='Balanced'
    )
    
    clf.fit(
        X_train, _y_train,
        eval_set=(X_val, _y_val),
        verbose=False
    )
    
    y_pred = clf.predict(X_val)
    print("Report for:", on)
    print(metrics.accuracy_score(_y_val, y_pred))
    print(metrics.balanced_accuracy_score(_y_val, y_pred))
    print(metrics.confusion_matrix(_y_val, y_pred))

for on in y_features:
    train_and_test(on, X_train, y_train, X_test, y_test, cat_features)

Report for: TWF
0.905
0.9522613065326633
[[1800  190]
 [   0   10]]
Report for: HDF
0.9945
0.9972194135490394
[[1967   11]
 [   0   22]]
Report for: PWF
0.9875
0.9936932391523714
[[1957   25]
 [   0   18]]
Report for: OSF
0.995
0.9974709155285786
[[1967   10]
 [   0   23]]
Report for: RNF
0.6275
0.563877755511022
[[1253  743]
 [   2    2]]
